In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import transforms , datasets
from PIL import Image
import pandas as pd
import re
import numpy as np
import random
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold



/home/jaskaran/.conda/envs/training/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not availabe")


CUDA is available!


In [3]:

class GeneticDataset(Dataset):
    def __init__(self,root_folder, transform =None):
        self.root_folder = root_folder
        self.transform = transform
        self.class_labels = sorted(os.listdir(root_folder))
        self.class_to_idx = {label: idx for idx ,label in enumerate(self.class_labels)}
        self.file_list = self._build_file_list()
    
    def _build_file_list(self):
        file_list = []
        for class_label in self.class_labels:
            class_path = os.path.join(self.root_folder,class_label)
            class_image = [os.path.join(class_path,image_path) for image_path in os.listdir(class_path)]
            file_list.extend(class_image)
        return file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,index):
        file_path = self.file_list[index]
        image = Image.open(file_path)
        image = image.convert("RGB")
        resized_image = image.resize((200,200))
        class_label = os.path.basename(os.path.dirname(file_path))
        label = self.class_to_idx[class_label]
        if self.root_folder.find("training") != -1:
            find = re.match(r".+?[_].+?[_].+?[_](.+?)[.][p][n][g]",file_path)
        else:
            find = re.match(r".+?[_].+?[_](.+?)[.][p][n][g]",file_path)
        if (self.transform) :
            resized_image = self.transform(resized_image)
        return {'image': resized_image, 'label': label, "coefficient" : float(find[1]) }

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


data = GeneticDataset("/home/jaskaran/data_images", transform= transform)
generator = torch.Generator().manual_seed(46)
length_data = len(data)
training_size = int(0.8*length_data)
testing_size = length_data - training_size

total_samples = 6000



indices = np.random.choice(len(data), total_samples, replace=False)

# Create a subset of the dataset using the random indices
subset_data = torch.utils.data.Subset(data, indices)




train_data , test_data = random_split(subset_data,[4800, 1200],generator=generator)

training_dataloader= DataLoader(train_data,batch_size=64,shuffle=True, drop_last=True)
testing_dataloader = DataLoader(test_data,batch_size=64,shuffle=True, drop_last=True)



In [5]:
train_data[1599]["image"].shape
len(train_data)

4800

In [6]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, 
                               out_channels=8,
                               kernel_size=7, 
                               stride= 1,
                               padding = 7)
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax()
        self.maxpool = nn.MaxPool2d(kernel_size =2,
                                    stride=2)
        #self.conv2 = nn.Conv2d(in_channels = 8,
                               #out_channels = 32,
                               #kernel_size = 2,
                               #stride = 1,
                               #padding = 2)
        self.fc1 = nn.Linear(86528,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,2)
        self.flatten = nn.Flatten(start_dim=1)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
       x = self.conv1(x)
       x = self.relu(x)
       x = self.maxpool(x)
    
       #x = self.conv2(x)
       #x = self.relu(x)
       #x = self.maxpool(x)
    
       x = self.flatten(x)
    
       x = self.fc1(x)
       x = self.relu(x)
       x = self.dropout(x)
    
       x = self.fc2(x)
       x = self.relu(x)
       x = self.dropout(x)
    
       x = self.fc3(x)
    
       return x


In [7]:
# Set seed for CPU operations
torch.manual_seed(45)

# Check if CUDA is available
if torch.cuda.is_available():
    # Set seed for CUDA operations
    torch.cuda.manual_seed(45)
    torch.cuda.manual_seed_all(45)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to("cuda:3")
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    
    for batch in training_dataloader:
        images = batch['image'].to("cuda:3")
        labels = batch['label'].to("cuda:3")  # Ensure labels are also moved to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in testing_dataloader:
            images = batch['image'].to("cuda:3")
            labels = batch['label'].to("cuda:3")
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total 
    print(f'Epoch {epoch + 1}/{num_epochs},loss : {loss:.2f} Accuracy: {accuracy:.2%}')

print("Model is trained")

Epoch 1/20,loss : 0.69 Accuracy: 48.44%


Epoch 2/20,loss : 0.72 Accuracy: 68.66%


Epoch 3/20,loss : 0.68 Accuracy: 48.35%


Epoch 4/20,loss : 0.38 Accuracy: 52.00%


Epoch 5/20,loss : 0.41 Accuracy: 90.28%


...

Epoch 16/20,loss : 0.15 Accuracy: 92.62%


Epoch 17/20,loss : 0.27 Accuracy: 94.70%


Epoch 18/20,loss : 0.45 Accuracy: 95.05%


Epoch 19/20,loss : 0.46 Accuracy: 93.40%


Epoch 20/20,loss : 0.23 Accuracy: 93.06%
Model is trained


In [8]:
#print(model.state_dict())
torch.save(model.state_dict(), f'model_state_dict_6000_1conv_0.001_7*7_8_filter_images.pth')